In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import functools
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Original paper of how to deal with structured data and csv data with tensorflow:
https://www.tensorflow.org/tutorials/load_data/csv  
This is just my experiment with drug200 dataset.

In [ ]:
plt.style.use('ggplot')
tf.get_logger().setLevel('ERROR')

### Data overview 

In [ ]:
data = pd.read_csv("../input/drug-classification/drug200.csv")

In [ ]:
data.head()

In [ ]:
data.shape[0]

In [ ]:
data.isnull().sum()

So, we have:
* Few observations (200)
* No missing values
* Two numerical features
* Three categorical features
* Categorical target column

### Creating tf dataset

In [ ]:
TARGET_COLUMN = "Drug"
BATCH_SIZE = 4

In [ ]:
target_labels = data.Drug.unique()

On the one hand, we can't just pass strings as features to the model. On the other hand, we don't want to make another csv dataset with encoded target column. So we create mapping function. In tensorflow we use StringLookup for this purposes.

In [ ]:
drugs_ind = tf.keras.layers.experimental.preprocessing.StringLookup(
                        vocabulary=list(target_labels), num_oov_indices=0, mask_token=None)

In [ ]:
ds = tf.data.experimental.make_csv_dataset(
      "../input/drug-classification/drug200.csv",
      batch_size=BATCH_SIZE,
      label_name=TARGET_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)

In [ ]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [ ]:
show_batch(ds)

Ok, let's define numerical and categorical features

In [ ]:
NUMERIC_FEATURES = ['Age','Na_to_K']

In [ ]:
CATEGORIES = {k: data[k].unique().tolist() for k in data if k not in NUMERIC_FEATURES}
CATEGORIES

In [ ]:
target_labels = CATEGORIES.pop("Drug")
target_labels

Now, we have to preprocess features

In [ ]:
class PreprocessFeatures(object):
    def __init__(self, names, labels_map):
        self.names = names
        self.labels_map = labels_map

    def __call__(self, features, labels):
        numeric_freatures = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        labels = self.labels_map(labels)
        return features, labels

In [ ]:
ds = ds.map(PreprocessFeatures(NUMERIC_FEATURES,drugs_ind))

### Handle numerical features

In [ ]:
desc = data[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

In [ ]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

In [ ]:
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

### Handle categorical features

In [ ]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

### Creating preprocessing layers

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

### Define model

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(len(target_labels), activation='sigmoid'),
])

In [ ]:
model.compile(
    loss=tf.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy())

### Train test split

In [ ]:
N = data.shape[0]
train_size = 0.8
n_train = int(N*train_size)
take_train = n_train // BATCH_SIZE

In [ ]:
train_ds = ds.take(take_train)
test_ds = ds.skip(take_train) 

### Fit the model

In [ ]:
history = model.fit(train_ds, 
                    validation_data=test_ds,
                    epochs=20)

In [ ]:
rows, cols = 1, 2
fig, axs = plt.subplots(rows, cols, figsize=(10,5))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].set_title('Loss')
axs[0].legend(['train_loss','val_loss'])

axs[1].plot(history.history['sparse_categorical_accuracy'])
axs[1].plot(history.history['val_sparse_categorical_accuracy'])
axs[1].set_title('Accuracy')
axs[1].legend(['Train accuracy','Test accuracy'])

In [ ]:
print("Pred: \t   True:")
for X, y in test_ds:
    pred = model(X)
    print(f"{tf.argmax(pred,axis=1)}  {y}")